In [1]:
import pandas as pd
from faker import Faker
import mysql.connector

In [2]:
df = pd.read_csv('coffee_sales.csv')
df['card'] = 'NO-' + df['card'].str[-4:]
df

,date,datetime,cash_type,card,money,coffee_name
0,2024-03-01,2024-03-01 10:15:50.520,card,NO-0001,38.70,Latte
1,2024-03-01,2024-03-01 12:19:22.539,card,NO-0002,38.70,Hot Chocolate
2,2024-03-01,2024-03-01 12:20:18.089,card,NO-0002,38.70,Hot Chocolate
3,2024-03-01,2024-03-01 13:46:33.006,card,NO-0003,28.90,Americano
4,2024-03-01,2024-03-01 13:48:14.626,card,NO-0004,38.70,Latte
...,...,...,...,...,...,...
1390,2024-08-27,2024-08-27 10:04:04.563,card,NO-0375,32.82,Latte
1391,2024-08-28,2024-08-28 12:08:18.772,card,NO-0551,27.92,Americano with Milk
1392,2024-08-28,2024-08-28 12:10:04.139,card,NO-0551,32.82,Cappuccino
1393,2024-08-28,2024-08-28 12:35:06.944,card,NO-0375,23.02,Americano


In [3]:
df.columns

Index(['date', 'datetime', 'cash_type', 'card', 'money', 'coffee_name'], dtype='object')

In [4]:
# convert the 'coffee_name' column to category type and assign to unique numbers

df['coffee_id'] = df['coffee_name'].astype('category').cat.codes
df

,date,datetime,cash_type,card,money,coffee_name,coffee_id
0,2024-03-01,2024-03-01 10:15:50.520,card,NO-0001,38.70,Latte,7
1,2024-03-01,2024-03-01 12:19:22.539,card,NO-0002,38.70,Hot Chocolate,6
2,2024-03-01,2024-03-01 12:20:18.089,card,NO-0002,38.70,Hot Chocolate,6
3,2024-03-01,2024-03-01 13:46:33.006,card,NO-0003,28.90,Americano,0
4,2024-03-01,2024-03-01 13:48:14.626,card,NO-0004,38.70,Latte,7
...,...,...,...,...,...,...,...
1390,2024-08-27,2024-08-27 10:04:04.563,card,NO-0375,32.82,Latte,7
1391,2024-08-28,2024-08-28 12:08:18.772,card,NO-0551,27.92,Americano with Milk,1
1392,2024-08-28,2024-08-28 12:10:04.139,card,NO-0551,32.82,Cappuccino,2
1393,2024-08-28,2024-08-28 12:35:06.944,card,NO-0375,23.02,Americano,0


In [5]:
# create a separate DataFrame for coffee_id and coffee_name 

coffee_master = pd.DataFrame({
    'coffee_name': df['coffee_name'].astype('category').cat.categories,  # get unique coffee names (categories)
    'coffee_id': range(len(df['coffee_name'].astype('category').cat.categories))  # generate corresponding coffee_id
})

coffee_master

,coffee_name,coffee_id
0,Americano,0
1,Americano with Milk,1
2,Cappuccino,2
3,Cocoa,3
4,Cortado,4
5,Espresso,5
6,Hot Chocolate,6
7,Latte,7


In [6]:
#test
print(df['coffee_name'].astype('category') , end ='\n\n\n\n')
print(df['coffee_name'].astype('category').cat.codes , end ='\n\n\n\n')
print(df['coffee_name'].astype('category').cat.categories)

0                     Latte
1             Hot Chocolate
2             Hot Chocolate
3                 Americano
4                     Latte
               ...         
1390                  Latte
1391    Americano with Milk
1392             Cappuccino
1393              Americano
1394    Americano with Milk
Name: coffee_name, Length: 1395, dtype: category
Categories (8, object): ['Americano', 'Americano with Milk', 'Cappuccino', 'Cocoa', 'Cortado', 'Espresso', 'Hot Chocolate', 'Latte']



0       7
1       6
2       6
3       0
4       7
       ..
1390    7
1391    1
1392    2
1393    0
1394    1
Length: 1395, dtype: int8



Index(['Americano', 'Americano with Milk', 'Cappuccino', 'Cocoa', 'Cortado',
       'Espresso', 'Hot Chocolate', 'Latte'],
      dtype='object')


In [7]:
# create customer table

customer_master = df[['card']].drop_duplicates()
customer_master

,card
0,NO-0001
1,NO-0002
3,NO-0003
4,NO-0004
5,NO-0005
...,...
1384,NO-0548
1387,NO-0549
1389,NO-0550
1391,NO-0551


In [8]:
# crate fake name from card id
fake = Faker()
customer_master['customer_name'] = customer_master['card'].apply(lambda x: fake.name())

# Drop rows where 'card' column has NaN values
customer_master = customer_master.dropna(subset=['card'])
customer_master

,card,customer_name
0,NO-0001,James Perry
1,NO-0002,Jennifer Cardenas
3,NO-0003,Alison Lee
4,NO-0004,Michael Graves
5,NO-0005,John Williams
...,...,...
1384,NO-0548,Sandra Lloyd
1387,NO-0549,Jonathan Ross
1389,NO-0550,Sarah Williams
1391,NO-0551,Kayla Greene


In [9]:
df = df.drop(columns=['coffee_name'])
df['coffee_id'] = df['coffee_id'].astype('int64')
df = df.where(pd.notnull(df), None)  # Replace NaN with None
df['datetime'] = pd.to_datetime(df['datetime'])  # Convert to datetime type
df

,date,datetime,cash_type,card,money,coffee_id
0,2024-03-01,2024-03-01 10:15:50.520,card,NO-0001,38.70,7
1,2024-03-01,2024-03-01 12:19:22.539,card,NO-0002,38.70,6
2,2024-03-01,2024-03-01 12:20:18.089,card,NO-0002,38.70,6
3,2024-03-01,2024-03-01 13:46:33.006,card,NO-0003,28.90,0
4,2024-03-01,2024-03-01 13:48:14.626,card,NO-0004,38.70,7
...,...,...,...,...,...,...
1390,2024-08-27,2024-08-27 10:04:04.563,card,NO-0375,32.82,7
1391,2024-08-28,2024-08-28 12:08:18.772,card,NO-0551,27.92,1
1392,2024-08-28,2024-08-28 12:10:04.139,card,NO-0551,32.82,2
1393,2024-08-28,2024-08-28 12:35:06.944,card,NO-0375,23.02,0


In [10]:
# we have 3 tables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395 entries, 0 to 1394
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1395 non-null   object        
 1   datetime   1395 non-null   datetime64[ns]
 2   cash_type  1395 non-null   object        
 3   card       1306 non-null   object        
 4   money      1395 non-null   float64       
 5   coffee_id  1395 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 65.5+ KB


In [11]:
customer_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 552 entries, 0 to 1394
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   card           552 non-null    object
 1   customer_name  552 non-null    object
dtypes: object(2)
memory usage: 12.9+ KB


In [12]:
coffee_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   coffee_name  8 non-null      object
 1   coffee_id    8 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 260.0+ bytes


In [13]:
# Insert into Database


In [14]:
# %%writefile .env
# MYSQL_HOST= ''
# MYSQL_PORT= 3306
# MYSQL_USER = ''
# MYSQL_PASSWORD = ''
# MYSQL_DB = ''

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
class Config:
    MYSQL_HOST = os.getenv("MYSQL_HOST")
    MYSQL_PORT = int(os.getenv("MYSQL_PORT"))
    MYSQL_USER = os.getenv("MYSQL_USER")
    MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
    MYSQL_DB = os.getenv("MYSQL_DB")


In [17]:
db = mysql.connector.connect(
    host= Config.MYSQL_HOST ,
    port =  Config.MYSQL_PORT ,
    user= Config.MYSQL_USER ,
    password= Config.MYSQL_PASSWORD,
    database = Config.MYSQL_DB
    )
mycursor = db.cursor()

In [18]:
# create transaction table
mycursor.execute("""
                    CREATE TABLE IF NOT EXISTS transaction (
                                                            date DATE,                 
                                                            datetime DATETIME,          
                                                            cash_type VARCHAR(255),
                                                            card VARCHAR(255),
                                                            money FLOAT,
                                                            coffee_id INT
                                                            );
                """)

In [19]:
# create customer table
mycursor.execute("CREATE TABLE IF NOT EXISTS customer (card VARCHAR(255) , customer_name VARCHAR(255));")

In [20]:
# create coffee table
mycursor.execute("CREATE TABLE IF NOT EXISTS coffee (coffee_name VARCHAR(255) , coffee_id INT);")

In [21]:
# Insert query into coffee table
insert_query_coffee = "INSERT INTO coffee (coffee_name, coffee_id) VALUES (%s, %s)"

# Insert data from the DataFrame row by row
for i, row in coffee_master.iterrows() :
    mycursor.execute(insert_query_coffee, tuple(row))

db.commit()

In [22]:
# Insert query into transaction table
insert_query_transaction = """ INSERT INTO transaction (date, datetime, cash_type, card, money, coffee_id)
                               VALUES (%s, %s, %s, %s, %s, %s)
                           """
chunk_size = 100   #insert 100 rows at a time

for i in range(0, len(df), chunk_size) :
    chunk = df[i : i + chunk_size] # Select the chunk
    
    data_tuples = [tuple(x) for x in chunk.values] #convert chunk into a list of tuples 
    
    mycursor.executemany(insert_query_transaction , data_tuples)  #execute the insert query for each chunk
    
    db.commit() #commit after each chunk

In [23]:
# Insert query into customer table
insert_query_customer = "INSERT INTO customer (card , customer_name) VALUES (%s, %s)"

chunk_size = 100   
for i in range(0, len(customer_master), chunk_size) :
    chunk = customer_master[i : i + chunk_size] 
    data_tuples = [tuple(x) for x in chunk.values] 
    mycursor.executemany(insert_query_customer , data_tuples)
    db.commit() 

In [24]:
#test
for i, row in coffee_master.iterrows():
    print(i , tuple(row))

0 ('Americano', 0)
1 ('Americano with Milk', 1)
2 ('Cappuccino', 2)
3 ('Cocoa', 3)
4 ('Cortado', 4)
5 ('Espresso', 5)
6 ('Hot Chocolate', 6)
7 ('Latte', 7)


In [25]:
# mycursor.execute("DROP TABLE IF EXISTS transaction")
# db.commit()
# mycursor.execute("DROP TABLE IF EXISTS customer")
# db.commit()
# mycursor.execute("DROP TABLE IF EXISTS coffee")
# db.commit()

In [26]:
# Query 
import warnings
warnings.filterwarnings('ignore')


query_01 = "SELECT * FROM coffee"
query_coffee = pd.read_sql(query_01, db)

'''
UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. 
Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
'''
query_coffee

,coffee_name,coffee_id
0,Americano,0
1,Americano with Milk,1
2,Cappuccino,2
3,Cocoa,3
4,Cortado,4
5,Espresso,5
6,Hot Chocolate,6
7,Latte,7


In [27]:
mycursor.execute("SELECT * FROM coffee")

for row in mycursor.fetchall() :
    print(row)

('Americano', 0)
('Americano with Milk', 1)
('Cappuccino', 2)
('Cocoa', 3)
('Cortado', 4)
('Espresso', 5)
('Hot Chocolate', 6)
('Latte', 7)


In [28]:
mycursor.execute("SELECT * FROM coffee")

for row in mycursor :
    print(row)

('Americano', 0)
('Americano with Milk', 1)
('Cappuccino', 2)
('Cocoa', 3)
('Cortado', 4)
('Espresso', 5)
('Hot Chocolate', 6)
('Latte', 7)


In [29]:
# Close the cursor and connection

# mycursor.close()
# db.close()

In [30]:
#save file
df.to_parquet("transaction.parquet", index=False)
coffee_master.to_parquet("coffee.parquet", index=False)
customer_master.to_parquet("customer.parquet", index=False)

In [31]:
import duckdb
connection = duckdb.connect()

In [32]:
duckdb.sql('SELECT * FROM transaction.parquet')

┌────────────┬─────────────────────────┬───────────┬─────────┬────────┬───────────┐
│    date    │        datetime         │ cash_type │  card   │ money  │ coffee_id │
│  varchar   │        timestamp        │  varchar  │ varchar │ double │   int64   │
├────────────┼─────────────────────────┼───────────┼─────────┼────────┼───────────┤
│ 2024-03-01 │ 2024-03-01 10:15:50.52  │ card      │ NO-0001 │   38.7 │         7 │
│ 2024-03-01 │ 2024-03-01 12:19:22.539 │ card      │ NO-0002 │   38.7 │         6 │
│ 2024-03-01 │ 2024-03-01 12:20:18.089 │ card      │ NO-0002 │   38.7 │         6 │
│ 2024-03-01 │ 2024-03-01 13:46:33.006 │ card      │ NO-0003 │   28.9 │         0 │
│ 2024-03-01 │ 2024-03-01 13:48:14.626 │ card      │ NO-0004 │   38.7 │         7 │
│ 2024-03-01 │ 2024-03-01 15:39:47.726 │ card      │ NO-0005 │   33.8 │         1 │
│ 2024-03-01 │ 2024-03-01 16:19:02.756 │ card      │ NO-0006 │   38.7 │         6 │
│ 2024-03-01 │ 2024-03-01 18:39:03.58  │ card      │ NO-0007 │   33.8 │     

In [33]:
duckdb.sql('SELECT count(*) FROM transaction.parquet')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1395 │
└──────────────┘

In [34]:
duckdb.sql(''' SELECT A.date , A.datetime , A.cash_type , A.card , C.customer_name , B.coffee_name , A.money
               FROM transaction.parquet A
               LEFT JOIN coffee.parquet B
               ON A.coffee_id  = B.coffee_id 
               LEFT JOIN customer.parquet C
               ON A.card = C.card;
           ''')

┌────────────┬─────────────────────────┬───────────┬─────────┬───────────────────┬─────────────────────┬────────┐
│    date    │        datetime         │ cash_type │  card   │   customer_name   │     coffee_name     │ money  │
│  varchar   │        timestamp        │  varchar  │ varchar │      varchar      │       varchar       │ double │
├────────────┼─────────────────────────┼───────────┼─────────┼───────────────────┼─────────────────────┼────────┤
│ 2024-03-01 │ 2024-03-01 10:15:50.52  │ card      │ NO-0001 │ James Perry       │ Latte               │   38.7 │
│ 2024-03-01 │ 2024-03-01 12:19:22.539 │ card      │ NO-0002 │ Jennifer Cardenas │ Hot Chocolate       │   38.7 │
│ 2024-03-01 │ 2024-03-01 12:20:18.089 │ card      │ NO-0002 │ Jennifer Cardenas │ Hot Chocolate       │   38.7 │
│ 2024-03-01 │ 2024-03-01 13:46:33.006 │ card      │ NO-0003 │ Alison Lee        │ Americano           │   28.9 │
│ 2024-03-01 │ 2024-03-01 13:48:14.626 │ card      │ NO-0004 │ Michael Graves    │ Latte

In [35]:
duckdb.sql(''' SELECT A.date , A.datetime , A.cash_type , A.card , C.customer_name , B.coffee_name , B.coffee_id
               FROM transaction.parquet A
               LEFT JOIN coffee.parquet B
               ON A.coffee_id  = B.coffee_id 
               LEFT JOIN customer.parquet C
               ON A.card = C.card
               WHERE cash_type = 'cash';
           ''')

┌────────────┬─────────────────────────┬───────────┬─────────┬───────────────┬─────────────────────┬───────────┐
│    date    │        datetime         │ cash_type │  card   │ customer_name │     coffee_name     │ coffee_id │
│  varchar   │        timestamp        │  varchar  │ varchar │    varchar    │       varchar       │   int64   │
├────────────┼─────────────────────────┼───────────┼─────────┼───────────────┼─────────────────────┼───────────┤
│ 2024-03-02 │ 2024-03-02 10:30:35.668 │ cash      │ NULL    │ NULL          │ Latte               │         7 │
│ 2024-03-03 │ 2024-03-03 10:10:43.981 │ cash      │ NULL    │ NULL          │ Latte               │         7 │
│ 2024-03-06 │ 2024-03-06 12:30:27.089 │ cash      │ NULL    │ NULL          │ Americano with Milk │         1 │
│ 2024-03-07 │ 2024-03-07 10:08:58.945 │ cash      │ NULL    │ NULL          │ Latte               │         7 │
│ 2024-03-07 │ 2024-03-07 11:25:43.977 │ cash      │ NULL    │ NULL          │ Latte            

In [36]:
#Query to find sales revenue from customers who paid for coffee by card, grouped by coffee type

duckdb.sql(''' SELECT B.coffee_name , SUM(CASE WHEN cash_type = 'card' THEN A.money ELSE 0 END) AS sales
               FROM transaction.parquet A
               LEFT JOIN coffee.parquet B
               ON A.coffee_id  = B.coffee_id 
               LEFT JOIN customer.parquet C
               ON A.card = C.card
               GROUP BY B.coffee_name
               ORDER BY B.coffee_name;
           ''')

┌─────────────────────┬────────────────────┐
│     coffee_name     │       sales        │
│       varchar       │       double       │
├─────────────────────┼────────────────────┤
│ Americano           │ 5061.2600000000175 │
│ Americano with Milk │   9982.49999999999 │
│ Cappuccino          │  7854.019999999983 │
│ Cocoa               │ 1499.9600000000005 │
│ Cortado             │  3446.820000000002 │
│ Espresso            │ 1212.1799999999992 │
│ Hot Chocolate       │  2736.400000000003 │
│ Latte               │  9888.879999999965 │
└─────────────────────┴────────────────────┘

In [37]:
#Query to find sales revenue(>100) from customers who paid for coffee by cash in may 2024, grouped by coffee type 

duckdb.sql(''' SELECT B.coffee_name , SUM(CASE WHEN cash_type = 'cash' THEN A.money ELSE 0 END) AS sales
               FROM transaction.parquet A
               LEFT JOIN coffee.parquet B
               ON A.coffee_id  = B.coffee_id 
               LEFT JOIN customer.parquet C
               ON A.card = C.card
               WHERE A.datetime BETWEEN '2024-04-30' AND '2024-06-01'
               GROUP BY B.coffee_name
               HAVING sales > 100
               ORDER BY B.coffee_name;

           ''')

┌─────────────────────┬────────┐
│     coffee_name     │ sales  │
│       varchar       │ double │
├─────────────────────┼────────┤
│ Americano           │  232.0 │
│ Americano with Milk │  136.0 │
│ Cappuccino          │  117.0 │
│ Latte               │  312.0 │
└─────────────────────┴────────┘

In [38]:
duckdb.sql(''' SELECT A.datetime , B.coffee_name , A.money
               FROM transaction.parquet A
               LEFT JOIN coffee.parquet B
               ON A.coffee_id  = B.coffee_id 
               LEFT JOIN customer.parquet C
               ON A.card = C.card
               -- WHERE A.datetime BETWEEN '2024-05-01' AND '2024-06-01'
               WHERE EXTRACT(MONTH FROM A.datetime) = 5 AND EXTRACT(DAY FROM A.datetime) = 2

           ''')

┌─────────────────────────┬─────────────────────┬────────┐
│        datetime         │     coffee_name     │ money  │
│        timestamp        │       varchar       │ double │
├─────────────────────────┼─────────────────────┼────────┤
│ 2024-05-02 10:33:55.746 │ Americano           │  27.92 │
│ 2024-05-02 18:36:22.138 │ Cappuccino          │  37.72 │
│ 2024-05-02 19:18:33.084 │ Latte               │  37.72 │
│ 2024-05-02 19:19:56.359 │ Cappuccino          │  37.72 │
│ 2024-05-02 13:49:39.898 │ Americano with Milk │  32.82 │
│ 2024-05-02 18:41:11.22  │ Americano with Milk │  32.82 │
│ 2024-05-02 11:29:57.08  │ Latte               │   39.0 │
└─────────────────────────┴─────────────────────┴────────┘